In [1]:
import os
#os.environ["CUDA_VISIBLE_DEVICES"]='0,1'

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.embeddings import HuggingFaceEmbeddings 
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma


device = "cuda" if torch.cuda.is_available() else "cpu"
#Need only 1 GPU if loading 8-bit model
print(device)

print("Using %d GPUs" %torch.cuda.device_count())

import gradio as gr
import time


/home/beams/MCHERUKARA/miniconda3/envs/HugFace/lib/python3.11/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (5.1.0)/charset_normalizer (3.1.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


cuda
Using 4 GPUs


In [2]:
model_name = "eachadea/vicuna-13b-1.1"
tokenizer_path = "./tokenizer/"

#Create a local tokenizer copy the first time
if os.path.isdir(tokenizer_path):
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
else:
    tokenizer = AutoTokenizer.from_pretrained("model_name")
    os.mkdir(tokenizer_path)
    tokenizer.save_pretrained(tokenizer_path)

In [3]:
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")#, load_in_8bit=True)
pipe = pipeline(
    "text-generation",
    model=model, 
    tokenizer=tokenizer, 
    max_length=1024,
    temperature=0.6,
    top_p=0.95,
    repetition_penalty=1.2
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
#Load embedding model and use that to embed text from source
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

with open("APS-Science-Highlight/2021-07-06_better-educated-neural-networks-for-nanoscale-3-d-coherent-x-ray.txt") as f:
    book = f.read()

print(book)

Better-Educated Neural Networks for Nanoscale 3-D Coherent X-ray Imaging

One of the inescapable realities of various imaging techniques is called the "phase problem," which simply refers to the loss of phase information inherent in the nature of imaging methods such as x-ray diffraction.  Though it might be inconvenient, it can be dealt with by using various mathematical methods to retrieve the phase data from the image with inverse computation.  Such methods, however, are not only time-consuming but require a great deal of computer power as they must run through multiple iterations to converge on a solution, which prevents real-time imaging.  A group of researchers working at the U.S. Department of Energy’s Advanced Photon Source (APS) has demonstrated a new approach to this perennial obstacle by using a deep-learning neural network trained and optimized for enhanced accuracy to perform fast three-dimensional (3-D) nanoscale imaging from coherent x-ray data.  The work was published i

In [5]:
text_splitter = CharacterTextSplitter(chunk_size=1024, chunk_overlap=0)
texts = text_splitter.split_text(book)
docsearch = Chroma.from_texts(
    texts, embeddings, metadatas=[{"source": str(i)} for i in range(len(texts))]
)

Using embedded DuckDB without persistence: data will be transient


In [27]:
query = "What is 3DCDINN?"
N_hits = 3
docs = docsearch.similarity_search_with_score(query, k=N_hits)
for doc in docs:
    print(doc)

(Document(page_content='The 3D-CDI-NN model begins with an offline training stage in which polyhedral shapes are created by randomly clipping an fcc crystal lattice structure, applying and relaxing various stresses in the Large-scale Atomic/Molecular Massively Parallel Simulator (LAMMPS) molecular dynamics program, and voxelizing the data into a 32 x 32 x 32 voxel grid.\xa0 The grid is then entered into 3D-CDI-NN and its predictions matched with the known solutions.\xa0 The 3D-CDI-NN framework is quite accurate in its predictions, even when working with under-sampled diffraction patterns.\xa0\n\nTo enhance the accuracy of 3D-CDI-NN even further, the investigators added a refinement that uses a reverse-mode automatic differentiation (AD) technique on the strain predictions and diffraction data.\xa0 This proved especially effective in recovering details of structural strain within the examined crystal samples, even when not using oversampled data, as is generally done when using AD techn

In [28]:
#Get context strings
context=""
for i in range(N_hits):
    context += docs[i][0].page_content +"\n"
print (context)

The 3D-CDI-NN model begins with an offline training stage in which polyhedral shapes are created by randomly clipping an fcc crystal lattice structure, applying and relaxing various stresses in the Large-scale Atomic/Molecular Massively Parallel Simulator (LAMMPS) molecular dynamics program, and voxelizing the data into a 32 x 32 x 32 voxel grid.  The grid is then entered into 3D-CDI-NN and its predictions matched with the known solutions.  The 3D-CDI-NN framework is quite accurate in its predictions, even when working with under-sampled diffraction patterns. 

To enhance the accuracy of 3D-CDI-NN even further, the investigators added a refinement that uses a reverse-mode automatic differentiation (AD) technique on the strain predictions and diffraction data.  This proved especially effective in recovering details of structural strain within the examined crystal samples, even when not using oversampled data, as is generally done when using AD techniques.
See: Henry Chan1,2*, Youssef S.

In [29]:

local_llm = HuggingFacePipeline(pipeline=pipe)


template = """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Context:
{context}

Current conversation:
{history}
Human: {input}
AI:"""

PROMPT = PromptTemplate(
    input_variables=["history", "input", "context"], template=template
)


memory = ConversationBufferWindowMemory(memory_key="history", 
                                        input_key = "input", 
                                        k=6)


conversation = LLMChain(
        prompt=PROMPT,
        llm=local_llm, 
        verbose=True, 
        memory=memory
)

#print(conversation.prompt.template)

In [30]:
conversation.predict(input=query, context=context)



> Entering new LLMChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Context:
The 3D-CDI-NN model begins with an offline training stage in which polyhedral shapes are created by randomly clipping an fcc crystal lattice structure, applying and relaxing various stresses in the Large-scale Atomic/Molecular Massively Parallel Simulator (LAMMPS) molecular dynamics program, and voxelizing the data into a 32 x 32 x 32 voxel grid.  The grid is then entered into 3D-CDI-NN and its predictions matched with the known solutions.  The 3D-CDI-NN framework is quite accurate in its predictions, even when working with under-sampled diffraction patterns. 

To enhance the accuracy of 3D-CDI-NN even further, the investigators added a refinement that uses a reverse-mode automatic differen

" 3D-CDI-NN stands for three-dimensional coherent diffractive imaging using neural networks. It's a deep learning method used to reconstruct images of nanostructures based on their diffraction patterns."